# AWS Service Data Use Case RAG + Titan ON Bedrock

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

Question Answering (QA) is an important task that involves extracting answers to factual queries posed in natural language. Typically, a QA system processes a query against a knowledge base containing structured or unstructured data and generates a response with accurate information. Ensuring high accuracy is key to developing a useful, reliable and trustworthy question answering system, especially for enterprise use cases. 

Generative AI models like Titan and Claude use probability distributions to generate responses to questions. These models are trained on vast amounts of text data, which allows them to predict what comes next in a sequence or what word might follow a particular word. However, these models are not able to provide accurate or deterministic answers to every question because there is always some degree of uncertainty in the data. Enterprises need to query domain specific and proprietary data and use the information to answer questions, and more generally data on which the model has not been trained on. 

In this module, we will demonstrate how to use the Bedrock Titan model to provide information response to queries.

In this example we will be running the Model with no context and then manually try and provide the context. There is no `RAG` augmentation happening here. This approach works with short documents or single-ton applications, it might not scale to enterprise level question answering where there could be large enterprise documents which cannot all be fit into the prompt sent to the model. 

### Challenges
- How to have the model return factual answers for question

### Proposal
To the above challenges, this notebook proposes the following strategy
#### Prepare documents
Before being able to answer the questions, the documents must be processed and a stored in a document store index
- Here we will send in the request with the full relevant context to the model and expect the response back


## Setup

Before running the rest of this notebook, you'll need to run the cells below to (ensure necessary libraries are installed and) connect to Bedrock.

For more details on how the setup works and ⚠️ **whether you might need to make any changes**, refer to the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb) notebook.

In [56]:
!pip install --upgrade pip


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [57]:
# Make sure you ran `download-dependencies.sh` from the root of the repository first!
%pip install --no-build-isolation --force-reinstall \
    ../dependencies/awscli-*-py3-none-any.whl \
    ../dependencies/boto3-*-py3-none-any.whl \
    ../dependencies/botocore-*-py3-none-any.whl

%pip install --quiet "faiss-cpu>=1.7,<2" langchain==0.0.249 "pypdf>=3.8,<4"

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Processing /root/dependencies/awscli-*-py3-none-any.whl
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/root/dependencies/awscli-*-py3-none-any.whl'

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [58]:
import os
from sagemaker import get_execution_role

role = get_execution_role()
print(role)

os.environ['BEDROCK_ASSUME_ROLE'] = role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
arn:aws:iam::218208277580:role/service-role/AmazonSageMaker-ExecutionRole-20230807T175994


In [59]:
import json
import os
import sys

import boto3

module_path = "."
sys.path.append(os.path.abspath(module_path))

print(os.path.abspath(module_path))
from utils import bedrock, print_ww

/root/amazon-bedrock-workshop


In [60]:
!pip install --upgrade boto3

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [61]:
import json
import os
import sys

import boto3

module_path = "."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


boto3_bedrock = bedrock.get_bedrock_client(
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## Section 1: Q&A with the knowledge of the model
In this section we try to use models provided by Bedrock service to answer questions based on the knowledge it gained during the training phase.

In this Notebook we will be using the `invoke_model()` method of Amazon Bedrock client. The mandatory parameters required to use this method are `modelId` which represents the Amazon Bedrock model ARN, and `body` which is the prompt for our task. The `body` prompt changes depending on the foundation model provider selected. We walk through this in detail below

```
{
   modelId= model_id,
   contentType= "application/json",
   accept= "application/json",
   body=body
}

```


## Configure langchain

We will begin to use langchain with titan

`llm = Bedrock(model_id="amazon.titan-tg1-large")`

Available model IDs include:

- `amazon.titan-tg1-large`
- `ai21.j2-grande-instruct`
- `ai21.j2-jumbo-instruct`
- `anthropic.claude-instant-v1`
- `anthropic.claude-v1`

In [62]:
import json
import os
import sys

import boto3

module_path = "."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww



boto3_bedrock = bedrock.get_bedrock_client(
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


In [63]:
!pip install FAISS-gpu

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


## Configure langchain

We begin with instantiating the LLM and the Embeddings model. Here we are using Anthropic Claude for text generation and Amazon Titan for text embedding.

Note: It is possible to choose other models available with Bedrock. You can replace the `model_id` as follows to change the model.

`llm = Bedrock(model_id="amazon.titan-tg1-large")`

Available model IDs include:

- `amazon.titan-tg1-large`
- `ai21.j2-grande-instruct`
- `ai21.j2-jumbo-instruct`
- `anthropic.claude-instant-v1`
- `anthropic.claude-v1`

In [64]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

# Here, we are creating our claude model v2
llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={'max_tokens_to_sample':2000, 'temperature':0})
bedrock_embeddings = BedrockEmbeddings(client=boto3_bedrock)

### Now, we will generate bedrock embeddings using the titan embeddings model!

In [65]:
import traceback
import time
from typing import Any, Dict, List, Optional
class BedrockEmbeddingsCustom(BedrockEmbeddings):
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Compute doc embeddings using a Bedrock model.

        Args:
            texts: The list of texts to embed

        Returns:
            List of embeddings, one for each text.
        """
        print(f"BedrockEmbeddingsCustom: embed_docs():: lenght of texts={len(texts)}::")
        results = []
        counter = 1
        errors = []
        for text in texts:
            try:
                response = self._embedding_func(text)
                results.append(response)
                #print(f"BedrockEmbeddingsCustom: embed_docs()::processed doc_{counter}:")
                counter+=1
            except:
                print(f"BedrockEmbeddingsCustom: ERROR ={traceback.format_exc()}:: WAITING for 20 SEC")
                time.sleep(20) # 20 sec
                errors.append(text)
        
        print(f"BedrockEmbeddingsCustom: embed_docs(): TRYING Errors now:len={len(errors)}:")
        for text in errors:
            print(f"BedrockEmbeddingsCustom: embed_docs(): error :text={text}:")
            try:
                response = self._embedding_func(text)
                results.append(response)
                #print(f"BedrockEmbeddingsCustom: embed_docs()::processed doc_{counter}:")
                counter+=1
            except:
                print(f"BedrockEmbeddingsCustom: ERROR ={text}:: WAITING for 20 SEC")
                time.sleep(20) # 20 sec
                    
        return results
    
bedrock_embeddings = BedrockEmbeddingsCustom(client=boto3_bedrock)
bedrock_embeddings

BedrockEmbeddingsCustom(client=<botocore.client.BedrockRuntime object at 0x7fe68f35f580>, region_name=None, credentials_profile_name=None, model_id='amazon.titan-e1t-medium', model_kwargs=None, endpoint_url=None)

## Preparing the Data: Chunk, embed and create vectors from your PDF data

In [46]:
!pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/f4/2e/0adf6e264b996e263b1c57cad6560ffd5492a69beb9fd779ed0463d486bc/tiktoken-0.5.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 88.9 MB/s eta 0:00:00


In [66]:
import glob
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

tiktoken_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=0
)
pdf_docs = [doc for pdf in glob.glob('dataAWS/*.pdf') for doc in PyPDFLoader(pdf).load_and_split(tiktoken_splitter)  ]

## Now, that we are splitting the documents using langchain, let's look at some stats

In [67]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
avg_char_count_pre = avg_doc_length(documents)
avg_char_count_post = avg_doc_length(pdf_docs)
print(f'Average length among {len(documents)} documents loaded is {avg_char_count_pre} characters.')
print(f'After the split we have {len(pdf_docs)} documents more than the original {len(documents)}.')
print(f'Average length among {len(pdf_docs)} documents (after split) is {avg_char_count_post} characters.')

Average length among 10 documents loaded is 2452 characters.
After the split we have 17 documents more than the original 10.
Average length among 17 documents (after split) is 1442 characters.


### From the data above, we have split the length across 34 documents and loaded into 2858 characters. Now, let's go ahead bedrock embeddings from these chunks of data that we have

In [73]:
prompt = """

Human: Please provide a summary of the following text.
<text>
Multi Account Strategy

Tagging 

* Use to organize resources. Helps with billing to break down costs, automation, and acces control.
* Tag Categories (technical tags, business tags, tags for automation, security tags).
* Best practices: standardized, case-sensitive, consistent across resources, leverage tools(resource groups).

Disaster Recovery

Disaster Recovery Strategies

* Pilot Light → DBs ready and data replication, create other resources when it’s time(VMs).
* Warm Standby → DBs ready and data replication, other resources operational but scaled down.
* The difference between Pilot Light and Warm Standby can sometimes be difficult to understand. Both include an environment in your DR Region with copies of your primary region assets. The distinction is that Pilot Light cannot process requests without additional action taken first, while Warm Standby can handle traffic (at reduced capacity levels) immediately.

Elastic Disaster Recovery

* Minimizes downtime and data loss with fast, reliable recovery of on-premises and cloud-based applications using affordable storage, minimal compute, and point-in-time recovery.
* Use cases: a) On-premises to AWS b) Cloud to AWS c) AWS Region to AWS Region

Identity and Access Management

AWS IAM Identity Center

* Supports only SAML 2.0–based applications so an OpenID Connect-compatible solution will not work.

Simple AD

* Subset of Microsoft AD.

Managed Microsoft AD

* Configure a trust relationship between AWS Managed Microsoft AD in the AWS Cloud and your existing on-premises Microsoft Active Directory, providing users and groups with access to resources in either domain, using single sign-on (SSO).

Microsoft AD connector

* In case you want to connect to on premises AD.

Identity Federation

* Identity federation with SAML 2.0 enables federated single sign-on (SSO). You can use a third-party SAML IdP to establish SSO access to the console or you can create a custom IdP to enable console access for your external users.
* Before you can use SAML 2.0-based federation, you must configure your organization’s IdP and your AWS account to trust each other. Inside your organization, you must have an IdP that supports SAML 2.0, like Microsoft Active Directory Federation Service (AD FS, part of Windows Server). In your organization’s IdP, you define assertions that map users or groups in your organization to the IAM roles.
* When you create the trust policy for the role, you specify the SAML provider that you created earlier as the Principal.
* Flow: LDAP → STS → AWS access. 
* Web Identity Federation with AssumeRoleWithWebIdentity API → STS → AWS resources.

Create SAML Identity provider in IAM

1. Get the SAML metadata document from IdP.
2. Create SAML IAM identity provider.
3. Configure SAML IdP with relying party trust.
4. In IdP, configure SAML Assertions for the authentication response.


</text>

Assistant:"""

Creating request body with prompt and inference parameters

Following the request syntax of invoke_model, you create request body with the above prompt and inference parameters.

In [74]:
body = json.dumps({"prompt": prompt,
                 "max_tokens_to_sample":4096,
                 "temperature":0.5,
                 "top_k":250,
                 "top_p":0.5,
                 "stop_sequences":[]
                  }) 

### Invoke foundation model via Boto3

Here sends the API request to Amazon Bedrock with specifying request parameters modelId, accept, and contentType. Following the prompt, the foundation model in Amazon Bedrock summarizes the text.

In [75]:
modelId = 'anthropic.claude-v2' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print_ww(response_body.get('completion'))

 Here is a summary of the key points from the text:

- Using multiple AWS accounts helps organize resources for billing, automation, and access control.
Best practices include standardized and consistent tagging.

- Disaster recovery strategies include Pilot Light (minimal always-on resources), Warm Standby
(scaled-down operational resources), and Elastic Disaster Recovery (for fast recovery using storage
and point-in-time recovery).

- AWS IAM options for identity and access management include Identity Center (SAML-based), Simple
AD, Managed Microsoft AD, and AD Connector.

- Identity federation with SAML enables single sign-on. Steps include getting SAML metadata from
IdP, creating IAM identity provider, configuring IdP trust, and configuring IdP assertions.


## PROMPT Engineering #1: Simple context giving and human assistant looping

In [77]:
prompt = """

Human: You are an AWS cloud advisor. Provide a helpful summary with actions users can take to focus on multi account strategy and IAM identity center. 
<text>
Multi Account Strategy

Tagging 

* Use to organize resources. Helps with billing to break down costs, automation, and acces control.
* Tag Categories (technical tags, business tags, tags for automation, security tags).
* Best practices: standardized, case-sensitive, consistent across resources, leverage tools(resource groups).

Disaster Recovery

Disaster Recovery Strategies

* Pilot Light → DBs ready and data replication, create other resources when it’s time(VMs).
* Warm Standby → DBs ready and data replication, other resources operational but scaled down.
* The difference between Pilot Light and Warm Standby can sometimes be difficult to understand. Both include an environment in your DR Region with copies of your primary region assets. The distinction is that Pilot Light cannot process requests without additional action taken first, while Warm Standby can handle traffic (at reduced capacity levels) immediately.

Elastic Disaster Recovery

* Minimizes downtime and data loss with fast, reliable recovery of on-premises and cloud-based applications using affordable storage, minimal compute, and point-in-time recovery.
* Use cases: a) On-premises to AWS b) Cloud to AWS c) AWS Region to AWS Region

Identity and Access Management

AWS IAM Identity Center

* Supports only SAML 2.0–based applications so an OpenID Connect-compatible solution will not work.

Simple AD

* Subset of Microsoft AD.

Managed Microsoft AD

* Configure a trust relationship between AWS Managed Microsoft AD in the AWS Cloud and your existing on-premises Microsoft Active Directory, providing users and groups with access to resources in either domain, using single sign-on (SSO).

Microsoft AD connector

* In case you want to connect to on premises AD.

Identity Federation

* Identity federation with SAML 2.0 enables federated single sign-on (SSO). You can use a third-party SAML IdP to establish SSO access to the console or you can create a custom IdP to enable console access for your external users.
* Before you can use SAML 2.0-based federation, you must configure your organization’s IdP and your AWS account to trust each other. Inside your organization, you must have an IdP that supports SAML 2.0, like Microsoft Active Directory Federation Service (AD FS, part of Windows Server). In your organization’s IdP, you define assertions that map users or groups in your organization to the IAM roles.
* When you create the trust policy for the role, you specify the SAML provider that you created earlier as the Principal.
* Flow: LDAP → STS → AWS access. 
* Web Identity Federation with AssumeRoleWithWebIdentity API → STS → AWS resources.

Create SAML Identity provider in IAM

1. Get the SAML metadata document from IdP.
2. Create SAML IAM identity provider.
3. Configure SAML IdP with relying party trust.
4. In IdP, configure SAML Assertions for the authentication response.


</text>

Follow the instructions below to summarize 
<instructions>
- The summary should incorporate analogies and examples for users.
- Add metaphorical examples to have users understand the importance of AWS.
- Make sure to add a full stop after each sentence
- Ask the user if you can help with anything else
</instructions>

Assistant:"""

In [78]:
body = json.dumps({"prompt": prompt,
                 "max_tokens_to_sample":4096,
                 "temperature":0.5,
                 "top_k":250,
                 "top_p":0.5,
                 "stop_sequences":[]
                  }) 

In [79]:
modelId = 'anthropic.claude-v2' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print_ww(response_body.get('completion'))

 Here is a helpful summary with analogies, examples, and actions for users to focus on multi-account
strategy and IAM identity center in AWS:

Managing multiple AWS accounts is like having separate bank accounts for your personal finances,
business finances, and savings. It helps you organize costs, control access, and mitigate risk. For
example, if your business account gets hacked, your personal account stays secure. Some good
practices are using clear naming conventions for accounts like "companyname-prod" and tagging
accounts for automation.

Identity and access management is like having security badges to enter different floors of an office
building. AWS IAM Identity Center is the security guard managing the badges. He only issues badges
compatible with magnetic stripe readers, so your fancy new RFID badge won’t work. You can get a
basic badge made quickly with Simple AD or a more advanced smart badge integrated with your existing
badges using Microsoft AD. If you have on-premises

## PROMPT ENGINEERING TECHNIQUE #3 - Now, let's focus on another technique to engineer this, by giving CLAUDE 'ROOM TO THINK'

In [83]:
prompt = """

Human: You are an AWS cloud advisor. Provide a helpful summary with actions users can take to focus on multi account strategy and IAM identity center. 
<text>
Multi Account Strategy

Tagging 

* Use to organize resources. Helps with billing to break down costs, automation, and acces control.
* Tag Categories (technical tags, business tags, tags for automation, security tags).
* Best practices: standardized, case-sensitive, consistent across resources, leverage tools(resource groups).

Disaster Recovery

Disaster Recovery Strategies

* Pilot Light → DBs ready and data replication, create other resources when it’s time(VMs).
* Warm Standby → DBs ready and data replication, other resources operational but scaled down.
* The difference between Pilot Light and Warm Standby can sometimes be difficult to understand. Both include an environment in your DR Region with copies of your primary region assets. The distinction is that Pilot Light cannot process requests without additional action taken first, while Warm Standby can handle traffic (at reduced capacity levels) immediately.

Elastic Disaster Recovery

* Minimizes downtime and data loss with fast, reliable recovery of on-premises and cloud-based applications using affordable storage, minimal compute, and point-in-time recovery.
* Use cases: a) On-premises to AWS b) Cloud to AWS c) AWS Region to AWS Region

Identity and Access Management

AWS IAM Identity Center

* Supports only SAML 2.0–based applications so an OpenID Connect-compatible solution will not work.

Simple AD

* Subset of Microsoft AD.

Managed Microsoft AD

* Configure a trust relationship between AWS Managed Microsoft AD in the AWS Cloud and your existing on-premises Microsoft Active Directory, providing users and groups with access to resources in either domain, using single sign-on (SSO).

Microsoft AD connector

* In case you want to connect to on premises AD.

Identity Federation

* Identity federation with SAML 2.0 enables federated single sign-on (SSO). You can use a third-party SAML IdP to establish SSO access to the console or you can create a custom IdP to enable console access for your external users.
* Before you can use SAML 2.0-based federation, you must configure your organization’s IdP and your AWS account to trust each other. Inside your organization, you must have an IdP that supports SAML 2.0, like Microsoft Active Directory Federation Service (AD FS, part of Windows Server). In your organization’s IdP, you define assertions that map users or groups in your organization to the IAM roles.
* When you create the trust policy for the role, you specify the SAML provider that you created earlier as the Principal.
* Flow: LDAP → STS → AWS access. 
* Web Identity Federation with AssumeRoleWithWebIdentity API → STS → AWS resources.

Create SAML Identity provider in IAM

1. Get the SAML metadata document from IdP.
2. Create SAML IAM identity provider.
3. Configure SAML IdP with relying party trust.
4. In IdP, configure SAML Assertions for the authentication response.


</text>

Follow the instructions below to summarize 
<instructions>
- The summary should incorporate analogies and examples for users.
- Add metaphorical examples to have users understand the importance of AWS.
- Make sure to add a full stop after each sentence
- Ask the user if you can help with anything else
</instructions>

Assistant: Can I think step by step?

Human: Yes. Think step by step and for each step, think out aloud by mentioning it in the <thinking> </thinking> xml tags to show what you are thinking. Give the summary in the <summary></summary> xml tags.

Assistant:"""

In [81]:
body = json.dumps({"prompt": prompt,
                 "max_tokens_to_sample":4096,
                 "temperature":0.5,
                 "top_k":250,
                 "top_p":0.5,
                 "stop_sequences":[]
                  }) 

In [82]:
modelId = 'anthropic.claude-v2' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print_ww(response_body.get('completion'))

 <thinking>
Okay, let's break this down:
First, I'll summarize the key points on multi-account strategy:
</thinking>

<summary>
To implement a solid multi-account strategy on AWS, it's important to use tagging to organize
resources and enable automation. For example, tagging servers by department like "Finance" or
"Engineering" helps track costs and control access. Standardized, consistent tagging is best
practice. It's also critical to plan for disaster recovery across accounts. Options like pilot
light, warm standby, and elastic disaster recovery balance cost, recovery time, and complexity. For
instance, a pilot light design keeps databases ready to replicate data, spinning up virtual machines
only when needed. This minimizes constant resource usage.
</summary>

<thinking>
Next, I'll cover the main points on IAM identity center:
</thinking>

<summary>
AWS IAM Identity Center is suited for SAML-based SSO, not OpenID Connect. For basic Active Directory
integration, AWS offers Simple AD

## PROMPT ENGINEERING TECHNIQUE #4 - Now, let's focus on another technique to engineer this, by giving CLAUDE A HUMAN ASSISTANT LOOP TO REINFORCE INSTRUCTIONS AND LET'S SEE HOW THE RESPONSE CHANGES

In [85]:
prompt = """

Human: You are an AWS cloud advisor. Provide a helpful summary with actions users can take to focus on multi account strategy and IAM identity center. 
<text>
Multi Account Strategy

Tagging 

* Use to organize resources. Helps with billing to break down costs, automation, and acces control.
* Tag Categories (technical tags, business tags, tags for automation, security tags).
* Best practices: standardized, case-sensitive, consistent across resources, leverage tools(resource groups).

Disaster Recovery

Disaster Recovery Strategies

* Pilot Light → DBs ready and data replication, create other resources when it’s time(VMs).
* Warm Standby → DBs ready and data replication, other resources operational but scaled down.
* The difference between Pilot Light and Warm Standby can sometimes be difficult to understand. Both include an environment in your DR Region with copies of your primary region assets. The distinction is that Pilot Light cannot process requests without additional action taken first, while Warm Standby can handle traffic (at reduced capacity levels) immediately.

Elastic Disaster Recovery

* Minimizes downtime and data loss with fast, reliable recovery of on-premises and cloud-based applications using affordable storage, minimal compute, and point-in-time recovery.
* Use cases: a) On-premises to AWS b) Cloud to AWS c) AWS Region to AWS Region

Identity and Access Management

AWS IAM Identity Center

* Supports only SAML 2.0–based applications so an OpenID Connect-compatible solution will not work.

Simple AD

* Subset of Microsoft AD.

Managed Microsoft AD

* Configure a trust relationship between AWS Managed Microsoft AD in the AWS Cloud and your existing on-premises Microsoft Active Directory, providing users and groups with access to resources in either domain, using single sign-on (SSO).

Microsoft AD connector

* In case you want to connect to on premises AD.

Identity Federation

* Identity federation with SAML 2.0 enables federated single sign-on (SSO). You can use a third-party SAML IdP to establish SSO access to the console or you can create a custom IdP to enable console access for your external users.
* Before you can use SAML 2.0-based federation, you must configure your organization’s IdP and your AWS account to trust each other. Inside your organization, you must have an IdP that supports SAML 2.0, like Microsoft Active Directory Federation Service (AD FS, part of Windows Server). In your organization’s IdP, you define assertions that map users or groups in your organization to the IAM roles.
* When you create the trust policy for the role, you specify the SAML provider that you created earlier as the Principal.
* Flow: LDAP → STS → AWS access. 
* Web Identity Federation with AssumeRoleWithWebIdentity API → STS → AWS resources.

Create SAML Identity provider in IAM

1. Get the SAML metadata document from IdP.
2. Create SAML IAM identity provider.
3. Configure SAML IdP with relying party trust.
4. In IdP, configure SAML Assertions for the authentication response.


</text>

Follow the instructions below to summarize 
<instructions>
- The summary should incorporate analogies and examples for users.
- Add metaphorical examples to have users understand the importance of AWS.
- Make sure to add a full stop after each sentence
- Ask the user if you can help with anything else
</instructions>

Assistant: Can I think step by step?

Human: Yes. Think step by step and for each step, think out aloud by mentioning it in the <thinking> </thinking> xml tags to show what you are thinking. Give the summary in the <summary></summary> xml tags.

Assistant: Is there anything else I can add in the answer?

Human: Yes. Teach the fundamentals in your answer like your user is a 5 year old and knows nothing about technology in the <5 year old understanding></5 year old understanding> xml tags.
- Give another section in your response where you teach the fundamentals in your answer like your user is a 50 year old tech CEO and knows everything about technology in the <50 year old CEO understanding></50 year old CEO understanding> xml tags.

Assistant:"""

In [86]:
body = json.dumps({"prompt": prompt,
                 "max_tokens_to_sample":4096,
                 "temperature":0.5,
                 "top_k":250,
                 "top_p":0.5,
                 "stop_sequences":[]
                  }) 

In [87]:
modelId = 'anthropic.claude-v2' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print_ww(response_body.get('completion'))

 <thinking>
Let me break this down step-by-step:
</thinking>

<summary>
Here is a helpful summary of key points about multi-account strategies and IAM identity center in
AWS:

- Use tagging to organize resources and enable automation, billing breakdown, and access control.
Standardize tags across accounts.
- Implement disaster recovery with pilot light, warm standby, or elastic disaster recovery. Choose
based on RTO/RPO.
- AWS IAM Identity Center supports SAML 2.0 apps. Use Simple AD, Microsoft AD, or AD Connector to
integrate on-prem AD.
- Enable SSO with SAML identity providers. Establish trust between IdP and AWS account. Define
assertions to map users/groups to IAM roles.
</summary>

<5 year old understanding>
AWS is like a big playground with lots of toys and games. You can use name tags to organize all the
toys into bins to find them easier later. If the playground gets messy from everyone playing, the
helpers can use the tags to clean up faster. The playground has a backup playg

## PROMPT ENGINEERING TECHNIQUE #5 - Now, LET'S FOCUS ON BREAKING THE PROMPT INTO SUB TASKS FOR YOUR USE CASE BELOW

In [88]:
prompt = """

Human: You are an AWS cloud advisor. Provide a helpful summary with actions users can take to focus on multi account strategy and IAM identity center. 
<text>
Multi Account Strategy

Tagging 

* Use to organize resources. Helps with billing to break down costs, automation, and acces control.
* Tag Categories (technical tags, business tags, tags for automation, security tags).
* Best practices: standardized, case-sensitive, consistent across resources, leverage tools(resource groups).

Disaster Recovery

Disaster Recovery Strategies

* Pilot Light → DBs ready and data replication, create other resources when it’s time(VMs).
* Warm Standby → DBs ready and data replication, other resources operational but scaled down.
* The difference between Pilot Light and Warm Standby can sometimes be difficult to understand. Both include an environment in your DR Region with copies of your primary region assets. The distinction is that Pilot Light cannot process requests without additional action taken first, while Warm Standby can handle traffic (at reduced capacity levels) immediately.

Elastic Disaster Recovery

* Minimizes downtime and data loss with fast, reliable recovery of on-premises and cloud-based applications using affordable storage, minimal compute, and point-in-time recovery.
* Use cases: a) On-premises to AWS b) Cloud to AWS c) AWS Region to AWS Region

Identity and Access Management

AWS IAM Identity Center

* Supports only SAML 2.0–based applications so an OpenID Connect-compatible solution will not work.

Simple AD

* Subset of Microsoft AD.

Managed Microsoft AD

* Configure a trust relationship between AWS Managed Microsoft AD in the AWS Cloud and your existing on-premises Microsoft Active Directory, providing users and groups with access to resources in either domain, using single sign-on (SSO).

Microsoft AD connector

* In case you want to connect to on premises AD.

Identity Federation

* Identity federation with SAML 2.0 enables federated single sign-on (SSO). You can use a third-party SAML IdP to establish SSO access to the console or you can create a custom IdP to enable console access for your external users.
* Before you can use SAML 2.0-based federation, you must configure your organization’s IdP and your AWS account to trust each other. Inside your organization, you must have an IdP that supports SAML 2.0, like Microsoft Active Directory Federation Service (AD FS, part of Windows Server). In your organization’s IdP, you define assertions that map users or groups in your organization to the IAM roles.
* When you create the trust policy for the role, you specify the SAML provider that you created earlier as the Principal.
* Flow: LDAP → STS → AWS access. 
* Web Identity Federation with AssumeRoleWithWebIdentity API → STS → AWS resources.

Create SAML Identity provider in IAM

1. Get the SAML metadata document from IdP.
2. Create SAML IAM identity provider.
3. Configure SAML IdP with relying party trust.
4. In IdP, configure SAML Assertions for the authentication response.


</text>

Follow the instructions below to summarize 
<instructions>
- The summary should incorporate analogies and examples for users.
- Add metaphorical examples to have users understand the importance of AWS.
- Make sure to add a full stop after each sentence
- Ask the user if you can help with anything else
</instructions>

Assistant: Can I think step by step?

Human: Yes. Think step by step and for each step, think out aloud by mentioning it in the <thinking> </thinking> xml tags to show what you are thinking. Give the summary in the <summary></summary> xml tags.

Assistant: Is there anything else I can add in the answer?

Human: Please follow these steps:
1. Write three topic sentences arguing for USING MULTI ACCOUNT AWS STRATEGIES.
2. Write three topic sentences arguing against not using AWS SERVICES OR IAM STRATEGIES.
3. Write an essay by expanding each topic sentence from Steps 1 and 2, and adding a conclusion to synthesize the arguments. Please enclose the essay in <essay></essay> tags.
4. Write your complete response like you are pitching your idea to a company

Assistant:"""

In [89]:
body = json.dumps({"prompt": prompt,
                 "max_tokens_to_sample":4096,
                 "temperature":0.5,
                 "top_k":250,
                 "top_p":0.5,
                 "stop_sequences":[]
                  }) 

In [90]:
modelId = 'anthropic.claude-v2' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print_ww(response_body.get('completion'))

 <thinking>
Okay, let's follow the instructions step-by-step:
</thinking>

<essay>

Using multi-account AWS strategies has several benefits. First, it improves security by separating
environments and access. Dev and prod environments can be isolated to prevent accidental changes.
Second, it aids compliance by restricting access and enabling granular privileges. Regulatory
requirements can be met more easily. Third, it reduces costs through consolidated billing and volume
discounts. Overall spend is optimized across accounts.

Not using AWS services or IAM strategies has downsides. First, security is weaker without the tools
AWS provides. Features like identity federation and fine-grained access controls are missing.
Second, compliance is more difficult lacking AWS's auditing capabilities. Demonstrating regulatory
adherence is harder. Third, costs are likely higher without AWS volume discounts and without ability
to right-size resources.

In conclusion, leveraging multi-account strategi

## PROMPT ENGINEERING TECHNIQUE #6- GIVE CONTEXT AND COMPREHENSION, PUT WORDS INTO CLAUDE'S MOUTH

In [91]:
prompt = """

Human: You are an AWS cloud advisor. Provide a helpful summary with actions users can take to focus on multi account strategy and IAM identity center. 
<text>
Multi Account Strategy

Tagging 

* Use to organize resources. Helps with billing to break down costs, automation, and acces control.
* Tag Categories (technical tags, business tags, tags for automation, security tags).
* Best practices: standardized, case-sensitive, consistent across resources, leverage tools(resource groups).

Disaster Recovery

Disaster Recovery Strategies

* Pilot Light → DBs ready and data replication, create other resources when it’s time(VMs).
* Warm Standby → DBs ready and data replication, other resources operational but scaled down.
* The difference between Pilot Light and Warm Standby can sometimes be difficult to understand. Both include an environment in your DR Region with copies of your primary region assets. The distinction is that Pilot Light cannot process requests without additional action taken first, while Warm Standby can handle traffic (at reduced capacity levels) immediately.

Elastic Disaster Recovery

* Minimizes downtime and data loss with fast, reliable recovery of on-premises and cloud-based applications using affordable storage, minimal compute, and point-in-time recovery.
* Use cases: a) On-premises to AWS b) Cloud to AWS c) AWS Region to AWS Region

Identity and Access Management

AWS IAM Identity Center

* Supports only SAML 2.0–based applications so an OpenID Connect-compatible solution will not work.

Simple AD

* Subset of Microsoft AD.

Managed Microsoft AD

* Configure a trust relationship between AWS Managed Microsoft AD in the AWS Cloud and your existing on-premises Microsoft Active Directory, providing users and groups with access to resources in either domain, using single sign-on (SSO).

Microsoft AD connector

* In case you want to connect to on premises AD.

Identity Federation

* Identity federation with SAML 2.0 enables federated single sign-on (SSO). You can use a third-party SAML IdP to establish SSO access to the console or you can create a custom IdP to enable console access for your external users.
* Before you can use SAML 2.0-based federation, you must configure your organization’s IdP and your AWS account to trust each other. Inside your organization, you must have an IdP that supports SAML 2.0, like Microsoft Active Directory Federation Service (AD FS, part of Windows Server). In your organization’s IdP, you define assertions that map users or groups in your organization to the IAM roles.
* When you create the trust policy for the role, you specify the SAML provider that you created earlier as the Principal.
* Flow: LDAP → STS → AWS access. 
* Web Identity Federation with AssumeRoleWithWebIdentity API → STS → AWS resources.

Create SAML Identity provider in IAM

1. Get the SAML metadata document from IdP.
2. Create SAML IAM identity provider.
3. Configure SAML IdP with relying party trust.
4. In IdP, configure SAML Assertions for the authentication response.


</text>

Follow the instructions below to summarize 
<instructions>
- The summary should incorporate analogies and examples for users.
- Add metaphorical examples to have users understand the importance of AWS.
- Make sure to add a full stop after each sentence
- Ask the user if you can help with anything else
</instructions>

Assistant: Is there anything else I can add in my answer?

Human: Yes, give me example of 5 AWS services for each of your argument.

Assistant: Yes, I understand. For a given sentence, I should specify the services for each of the instruction above and mention each servive in a bullet point like "[1]".

Human: Correct. Make sure to follow all these instructions.

Assistant:"""

In [92]:
body = json.dumps({"prompt": prompt,
                 "max_tokens_to_sample":4096,
                 "temperature":0.5,
                 "top_k":250,
                 "top_p":0.5,
                 "stop_sequences":[]
                  }) 

In [93]:
modelId = 'anthropic.claude-v2' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print_ww(response_body.get('completion'))

 Here is an updated summary with examples of AWS services for each point:

Multi Account Strategy

Tagging

* Use to organize resources like [1] EC2, [2] S3 buckets, [3] RDS databases. Helps with billing to
break down costs for [4] Lambda and [5] CloudFront usage, automation through [6] CloudFormation, and
access control via [7] IAM policies.

Disaster Recovery

Disaster Recovery Strategies

* Pilot Light → DBs ready and data replication with [1] RDS, create other resources like [2] EC2
instances when it’s time.
* Warm Standby → DBs ready and data replication with [3] DynamoDB, other resources like [4] ELB load
balancers operational but scaled down.

Elastic Disaster Recovery

* Minimizes downtime and data loss with fast, reliable recovery of on-premises and cloud-based
applications using [1] S3 for affordable storage, [2] EC2 for minimal compute, and [3] EBS for
point-in-time recovery.

Identity and Access Management

AWS IAM Identity Center

* Supports only [1] Cognito user pools for

## PROMPT ENGINEERING TECHNIQUE #7- Ask Claude for rewrites

In [94]:
prompt = """

Human: Here's a paragraph:
<paragraph>
Here is an updated summary with examples of AWS services for each point:

Multi Account Strategy

Tagging

* Use to organize resources like [1] EC2, [2] S3 buckets, [3] RDS databases. Helps with billing to
break down costs for [4] Lambda and [5] CloudFront usage, automation through [6] CloudFormation, and
access control via [7] IAM policies.

Disaster Recovery

Disaster Recovery Strategies

* Pilot Light → DBs ready and data replication with [1] RDS, create other resources like [2] EC2
instances when it’s time.
* Warm Standby → DBs ready and data replication with [3] DynamoDB, other resources like [4] ELB load
balancers operational but scaled down.

Elastic Disaster Recovery

* Minimizes downtime and data loss with fast, reliable recovery of on-premises and cloud-based
applications using [1] S3 for affordable storage, [2] EC2 for minimal compute, and [3] EBS for
point-in-time recovery.

Identity and Access Management

AWS IAM Identity Center

* Supports only [1] Cognito user pools for SAML 2.0–based applications so an [2] OpenID Connect-
compatible solution like Auth0 will not work.

Simple AD

* Subset of Microsoft AD using [1] Directory Service.

Managed Microsoft AD

* Configure a trust relationship between [1] AWS Managed Microsoft AD in the AWS Cloud and your
existing on-premises Microsoft Active Directory, providing users and groups with access to resources
in either domain, using [2] single sign-on (SSO) with AD Connector.

Microsoft AD connector

* In case you want to connect to on premises AD using [1] AD Connector.

Identity Federation

* Identity federation with [1] IAM SAML 2.0 enables federated single sign-on (SSO). You can use a
third-party SAML IdP like [2] PingFederate to establish SSO access to the console or you can create
a custom IdP to enable console access for your external users.

Does this help summarize the key points with relevant AWS service examples? Please let me know if
you need any clarification or have additional requirements.
</paragraph>

I'd like you to rewrite it using the following instructions:
<instructions>
- The summary should incorporate analogies and examples for users.
- Add metaphorical examples to have users understand the importance of AWS.
- Make sure to add a full stop after each sentence
- Ask the user if you can help with anything else
</instructions>

Assistant: Is there anything else I can add in my answer?

Human: Yes, give me example of 5 AWS services for each of your argument.

Assistant: Yes, I understand. For a given sentence, I should specify the services for each of the instruction above and mention each servive in a bullet point like "[1]".

Human: Correct. Make sure to follow all these instructions.

Please put your rewrite in <rewrite></rewrite> tags.

Assistant: <rewrite>

Assistant:"""

In [95]:
body = json.dumps({"prompt": prompt,
                 "max_tokens_to_sample":4096,
                 "temperature":0.5,
                 "top_k":250,
                 "top_p":0.5,
                 "stop_sequences":[]
                  }) 

In [96]:
modelId = 'anthropic.claude-v2' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print_ww(response_body.get('completion'))

 Here is an updated summary with relatable examples for users to understand the importance of AWS
services:

Multi Account Strategy is like having separate bank accounts for your personal, business, and
vacation funds. This helps organize your money and control spending, just like using services such
as [1] EC2, [2] S3, [3] RDS, [4] Lambda, and [5] CloudFront with tagging helps manage AWS resources
and billing.

Disaster Recovery is like having a backup generator during a blackout. AWS provides options to get
systems back up quickly:

Disaster Recovery Strategies

* Pilot Light is like keeping the pilot light on for your water heater, so it's ready to go. RDS and
data replication keep databases ready, while [1] EC2 instances can spin up when needed.
* Warm Standby is like keeping a spare tire inflated and ready to swap. [2] DynamoDB databases are
available, while [3] ELB load balancers and [4] EC2 instances are operational but scaled down.

Elastic Disaster Recovery

* Minimizes downti